In [13]:
import numpy as np
import pandas as pd
import os
import math as m
import matplotlib.pyplot as plt
import time

## Import Data Functions

In [14]:
def import_dataset(path):
    df = pd.read_csv(path, sep=';')
    df.columns = ["Name", "Cost","S1","S2","S3","S4","S5","S6","S7","S8","S9","S10","S11","S12","S13","S14","S15","S16","S17", "S18"]
    return df

def reduce_size(df_tot):
    for col in df_tot.columns :
        if col not in ["Configuration", "Instance", "Solution", "Cost"]:
            df_tot[col] = pd.to_numeric(df_tot[col], errors="ignore", downcast='float')
        else : 
            df_tot[col] = pd.to_numeric(df_tot[col], errors="ignore", downcast='integer')
    return df_tot

def tirage_repartition(indice, nb):
    tirage = []
    bond = int((len(indice)-1)/(nb-1))
    for k in range(0, len(indice), bond):
        tirage.append(indice[k])
    
    for element in tirage:
        indice.remove(element)

    return tirage, indice
    
def create_samples(p_train, nb_train, nb_test, nb_verify, seed):
    # Mettre la même seed pour avoir des résultats constants
    np.random.seed(seed)

    # Création des variables utiles
    fichiers_2113 = os.listdir('../HGS/2113/')
    fichiers_2213 = os.listdir('../HGS/2213/')
    fichiers_3113 = os.listdir('../HGS/3113/')
    fichiers_3213 = os.listdir('../HGS/3213/')

    #Shuffle 
    np.random.shuffle(fichiers_2113)
    np.random.shuffle(fichiers_2213)
    np.random.shuffle(fichiers_3113)
    np.random.shuffle(fichiers_3213)

    #Choix des fichiers trains et test (On arrondie au supérieur)
    train_list = {}
    test_list = {}
    train_list["2113"], test_list["2113"] = fichiers_2113[:m.ceil(p_train*len(fichiers_2113))], fichiers_2113[m.ceil(p_train*len(fichiers_2113)):]
    train_list["2213"], test_list["2213"] = fichiers_2213[:m.ceil(p_train*len(fichiers_2213))], fichiers_2213[m.ceil(p_train*len(fichiers_2213)):]
    train_list["3113"], test_list["3113"] = fichiers_3113[:m.ceil(p_train*len(fichiers_3113))], fichiers_3113[m.ceil(p_train*len(fichiers_3113)):]
    train_list["3213"], test_list["3213"] = fichiers_3213[:m.ceil(p_train*len(fichiers_3213))], fichiers_3213[m.ceil(p_train*len(fichiers_3213)):]

    #On créer les trois data_frames
    colonnes = ["Configuration", "Instance", "Solution", "Cost","S1","S2","S3","S4","S5","S6","S7","S8","S9","S10","S11","S12","S13","S14","S15","S16","S17", "S18"]
    lignes_train = []
    lignes_verify = []
    lignes_test = []


    for instance in train_list.keys():
        fichiers_train = train_list[instance]
        fichiers_test = test_list[instance]

        for path_fichier in fichiers_train:
            df = import_dataset(f"../HGS/{instance}/" + path_fichier)
            path_fichier = path_fichier.split("_")
            num = path_fichier[-1].replace(".csv", "")

            #On trie le dataset pour avoir un échantillon représentatif 
            df = df.sort_values(by = "Cost", ascending=False)
            df = df.reset_index().drop(columns=["index"])


            indice = [k for k in range(df.shape[0])]
            tirage_train, indice = tirage_repartition(indice, nb_train)
            tirage_verify, indice = tirage_repartition(indice, nb_verify)

            for i in range(nb_train):
                row_index = tirage_train.pop(0)
                row = list(df.iloc[row_index])
                row.pop(0)
                row = [int(instance), int(num), row_index] + row 
                lignes_train.append(row)
                
            for i in range(nb_verify):
                row_index = tirage_verify.pop(0)
                row = list(df.iloc[row_index])
                row.pop(0)
                row = [int(instance), int(num), row_index] + row 
                lignes_verify.append(row)
                

        for path_fichier in fichiers_test:
            df = import_dataset(f"../HGS/{instance}/" + path_fichier)
            path_fichier = path_fichier.split("_")
            num = path_fichier[-1].replace(".csv", "")

            #On trie le dataset pour avoir un échantillon représentatif 
            df = df.sort_values(by = "Cost", ascending=False)
            df = df.reset_index().drop(columns=["index"])
            indice = [k for k in range(df.shape[0])]
            tirage_test, indice = tirage_repartition(indice, nb_test)
            tirage_verify, indice = tirage_repartition(indice, nb_verify)

            for i in range(nb_test):
                row_index = tirage_test.pop(0)
                row = list(df.iloc[row_index])
                row.pop(0)
                row = [int(instance), int(num), row_index] + row 
                lignes_test.append(row)
            for i in range(nb_verify):
                row_index = tirage_verify.pop(0)
                row = list(df.iloc[row_index])
                row.pop(0)
                row = [int(instance), int(num), row_index] + row 
                lignes_verify.append(row)
        


    #Création des data frames

    df_train = pd.DataFrame(lignes_train, columns=colonnes)
    df_verify = pd.DataFrame(lignes_verify, columns=colonnes)
    df_test = pd.DataFrame(lignes_test, columns=colonnes)

    df_train = df_train.drop(columns=['S7'])
    df_verify = df_verify.drop(columns=['S7'])
    df_test = df_test.drop(columns=['S7'])
    return reduce_size(df_train), reduce_size(df_verify), reduce_size(df_test)


## Datasets and Scaling

In [15]:
df_train, df_verify, df_test = create_samples(0.8, 10000, 1000, 1000, 1)

In [16]:
X_train, Y_train = df_train.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_train['Cost']
X_verify, Y_verify = df_verify.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_verify['Cost']
X_test, Y_test = df_test.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_test['Cost']

In [17]:
from sklearn.preprocessing import StandardScaler

X_train_f = StandardScaler().fit(X_train).transform(X_train)
Y_train_f = StandardScaler().fit(Y_train.values.reshape(-1,1)).transform(Y_train.values.reshape(-1,1))

X_verify_f = StandardScaler().fit(X_verify).transform(X_verify)
Y_verify_f = StandardScaler().fit(Y_verify.values.reshape(-1,1)).transform(Y_verify.values.reshape(-1,1))

X_test_f = StandardScaler().fit(X_test).transform(X_test)
Y_test_f = StandardScaler().fit(Y_test.values.reshape(-1,1)).transform(Y_test.values.reshape(-1,1))

print("Done")

Done


## Model

In [20]:
from sklearn.neighbors import KNeighborsRegressor

start = time.time()
regr = KNeighborsRegressor(n_neighbors=4)
regr.fit(X_train_f, Y_train_f)
end = time.time()
elapsed = end - start

print("Temps exécution : ", elapsed)

Temps exécution :  0.07699871063232422


In [19]:
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import mean_absolute_error

#Get the time of execution
start = time.time()


score_mape = mean_absolute_percentage_error(Y_test_f, regr.predict(X_test_f))
score_mae = mean_absolute_error(Y_test_f, regr.predict(X_test_f))


end = time.time()
elapsed = end - start

print("Temps exécution : ", elapsed)
print("MAPE : ", score_mape)
print("MAE : ", score_mae)

Temps exécution :  1086.0301818847656
MAPE :  0.3844596841141698
MAE :  0.2639730443035203


Cross validation

In [21]:
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

#Get the time of execution
start = time.time()

scorer = make_scorer(mean_absolute_percentage_error)
score = cross_val_score(regr, X_train_f, Y_train_f, cv=10, scoring=scorer)


end = time.time()
elapsed = end - start

print("Temps exécution : ", elapsed)
print("%0.2f accuracy with a standard deviation of %0.2f" % (score.mean(), score.std()))

Temps exécution :  474.5206470489502
0.29 accuracy with a standard deviation of 0.33


## Tuning some shit

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import make_scorer

score_test = []
score_verify = []
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

for n_neighbors in range(2, 10):
    neigh = KNeighborsRegressor(n_neighbors=n_neighbors)
    neigh.fit(X_train_f, Y_train_f)
    score_test.append(mean_absolute_percentage_error(Y_test_f, regr.predict(X_test_f)))
    score_verify.append(mean_absolute_percentage_error(Y_verify_f, regr.predict(X_verify_f)))

print(score_test, score_verify)

In [ ]:
plt.plot([k for k in range(2,10)], score_test, label='Test')
plt.plot([k for k in range(2,10)], score_verify, label='Verify')

## HomeMade CrossValidation

In [ ]:
def training(seed):
    df_train, df_verify, df_test = create_samples(0.8, 10000, 1000, 1000, seed)
    X_train, Y_train = df_train.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_train['Cost']
    X_verify, Y_verify = df_verify.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_verify['Cost']
    X_test, Y_test = df_test.drop(columns=["Configuration", "Instance", "Solution", "Cost"]), df_test['Cost']

    X_train_f = StandardScaler().fit(X_train).transform(X_train)
    Y_train_f = StandardScaler().fit(Y_train.values.reshape(-1,1)).transform(Y_train.values.reshape(-1,1))

    X_test_f = StandardScaler().fit(X_test).transform(X_test)
    Y_test_f = StandardScaler().fit(Y_test.values.reshape(-1,1)).transform(Y_test.values.reshape(-1,1))

    regr = KNeighborsRegressor(n_neighbors=4)
    regr.fit(X_train_f, Y_train_f)
    score_mape = mean_absolute_percentage_error(Y_test_f, regr.predict(X_test_f))
    score_mae = mean_absolute_error(Y_test_f, regr.predict(X_test_f))


    return score_mape, score_mae



scores_mape, scores_mae = [], []


for seed in range(10):
    score_mape, score_mae = training(seed)
    scores_mape.append(score_mape)
    scores_mae.append(score_mae)



print("Mean scores mape : " , np.mean(scores_mape))
print("Mean scores mae : " , np.mean(scores_mae))
    